In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,...,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_100k
67,ABW,Aruba,2020-05-29,101,0,3,0,945.994,0.000,28.099,...,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
208,AFG,Afghanistan,2020-05-29,13036,580,235,8,334.872,14.899,6.037,...,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5
277,AGO,Angola,2020-05-29,73,2,4,0,2.221,0.061,0.122,...,2.405,1.362,5819.495,NaN,276.045,3.94,NaN,NaN,26.664,NaN


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-05-28,France,146011.0,0.0,28691.0,0.0,0.0,0.0
0,2020-05-28,China,84169.0,0.0,4654.0,0.0,0.0,0.0
0,2020-05-28,Italy,231648.0,0.0,33168.0,0.0,0.0,0.0
0,2020-05-28,Spain,237618.0,0.0,27327.0,0.0,0.0,0.0
0,2020-05-28,United States,1721026.0,0.0,101275.0,0.0,0.0,0.0
0,2020-05-28,World,5767025.0,0.0,359764.0,0.0,0.0,0.0
0,2020-05-28,United Kingdom,271445.0,0.0,37653.0,0.0,0.0,0.0
0,2020-05-28,Germany,180149.0,0.0,8459.0,0.0,0.0,0.0
0,2020-05-28,Iran,143531.0,0.0,7628.0,0.0,0.0,0.0
0,2020-05-28,Turkey,159634.0,0.0,4421.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-05-29,France,146011.0,149071.0,28691.0,28662.0,-3060.0,29.0
0,2020-05-29,China,84169.0,84106.0,4654.0,4638.0,63.0,16.0
0,2020-05-29,Italy,231648.0,231732.0,33168.0,33142.0,-84.0,26.0
0,2020-05-28,Spain,237618.0,237906.0,27327.0,27119.0,-288.0,208.0
0,2020-05-29,United States,1721026.0,1721750.0,101275.0,101617.0,-724.0,-342.0
0,2020-05-29,World,5767025.0,5776934.0,359764.0,360089.0,-9909.0,-325.0
0,2020-05-29,United Kingdom,271445.0,269127.0,37653.0,37837.0,2318.0,-184.0
0,2020-05-29,Germany,180149.0,180458.0,8459.0,8450.0,-309.0,9.0
0,2020-05-29,Iran,143531.0,143849.0,7628.0,7627.0,-318.0,1.0
0,2020-05-29,Turkey,159634.0,160979.0,4421.0,4461.0,-1345.0,-40.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")